# Imports and Datasets Preparations

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, r2_score, mean_squared_error
from pprint import pprint

## Boston Housing Dataset for Regression

In [9]:
boston_housing = pd.read_csv('datasets/boston_housing/BostonHousing.csv')
boston_housing.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [10]:
boston_housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   crim     506 non-null    float64
 1   zn       506 non-null    float64
 2   indus    506 non-null    float64
 3   chas     506 non-null    int64  
 4   nox      506 non-null    float64
 5   rm       506 non-null    float64
 6   age      506 non-null    float64
 7   dis      506 non-null    float64
 8   rad      506 non-null    int64  
 9   tax      506 non-null    int64  
 10  ptratio  506 non-null    float64
 11  b        506 non-null    float64
 12  lstat    506 non-null    float64
 13  medv     506 non-null    float64
dtypes: float64(11), int64(3)
memory usage: 55.5 KB


In [11]:
training_data_reg = boston_housing.sample(frac=0.7, random_state=1234)
data_without_train_reg = boston_housing.drop(training_data_reg.index)
validation_data_reg = data_without_train_reg.sample(frac=0.5, random_state=27)
test_data_reg = data_without_train_reg.drop(validation_data_reg.index)

In [12]:
print(f'Dataset split:\ntraining data: {(len(training_data_reg))}\nvalidation data: {(len(validation_data_reg))}\ntest data: {(len(test_data_reg))}')

Dataset split:
training data: 354
validation data: 76
test data: 76


### Features and Target preparation

In [13]:
# target training and validation
                
yreg = training_data_reg['medv'] # Target of training set
yreg_val = validation_data_reg['medv'] # Target of validation set
yreg.head()

64     33.0
100    27.5
400     5.6
485    21.2
454    14.9
Name: medv, dtype: float64

In [14]:
# the features vector

features_reg = ['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat']

Xreg = training_data_reg[features_reg]
Xreg_val = validation_data_reg[features_reg]
Xreg.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat
64,0.01951,17.5,1.38,0,0.4161,7.104,59.5,9.2229,3,216,18.6,393.24,8.05
100,0.14866,0.0,8.56,0,0.5200,6.727,79.9,2.7778,5,384,20.9,394.76,9.42
400,25.04610,0.0,18.10,0,0.6930,5.987,100.0,1.5888,24,666,20.2,396.90,26.77
485,3.67367,0.0,18.10,0,0.5830,6.312,51.9,3.9917,24,666,20.2,388.62,10.58
454,9.51363,0.0,18.10,0,0.7130,6.728,94.1,2.4961,24,666,20.2,6.68,18.71


# Bayesian Ridge Regression

## Bayesian Regression
Before we need to introduce the idea of the Bayesian Regression. In the Bayesian, we formulate linear regression using probability distributions rather than point estimates. The target y, is not estimated as a single value, but it is assumed to be drawn from a probabilistic distribution. So each y is seen as a sample, which is sampled from a normal distribution, where the normal distribution parameters are treated as random variable which are estimated from the data:
$$
y = \mathcal{N}(W^T X,\,\sigma^{2}I)
$$
Where:
- the mean: $W^T$ weight multiplied by the data points
- the variance is the square of the standard deviation multiplied by the Identity matrix (because we have multidimensional data)
<br>
The aim of Bayesian Linear Regression is not to find the single “best” value of the model parameters, but rather to determine the posterior distribution for the model parameters. Using the Bayesian' rules:
$$
p(W | data) = \frac{p(data | W) * p(W)}{p(data)}
$$
where:
- $p(W | data)$ is the posterior probability of parameters W given training data
- $p(data | W)$ is the probability (likelihood) of observing D given W
- $p(W)$ is the prior probability over the parameters
- $p(data)$ is the marginal likelihood (normalizing constant)

We want the most probable value of W given the data: maximum a posteriori (MAP). It is the mode of the posterior.
<br>
To clarify the idea:
- Priors: If we have domain knowledge, or a guess for what the model parameters should be, we can include them in our model, unlike in the frequentist approach which assumes everything there is to know about the parameters comes from the data. If we don’t have any estimates ahead of time, we can use non-informative priors for the parameters such as a normal distribution.
- Posterior: The result of performing Bayesian Linear Regression is a distribution of possible model parameters based on the data and the prior. This allows us to quantify our uncertainty about the model: if we have fewer data points, the posterior distribution will be more spread out.

As the amount of data points increases, the likelihood washes out the prior, and in the case of infinite data, the outputs for the parameters converge to the values obtained from OLS.
<br>
Here a visual examples:
<center><img src="images/brr/blr.png" width="450" center=/></center>

## Bayesian Ridge
Bayesian Ridge estimates a probabilistic model of the regression problem as described above but the prior for the coefficient is given by a spherical Gaussian, so used the idea of the ridge regression instead only of the linear regression.
The ridge minimize the classical linear least squared adding the l2 penalty.

In [17]:
from sklearn.linear_model import BayesianRidge

brr_model = BayesianRidge()
pprint(brr_model.get_params())

{'alpha_1': 1e-06,
 'alpha_2': 1e-06,
 'alpha_init': None,
 'compute_score': False,
 'copy_X': True,
 'fit_intercept': True,
 'lambda_1': 1e-06,
 'lambda_2': 1e-06,
 'lambda_init': None,
 'n_iter': 300,
 'normalize': 'deprecated',
 'tol': 0.001,
 'verbose': False}


## Main Parameters

The main parameters for Bayesian Ridge Regression are:
- n_iter: max number of iteration
- tol: convergence parameter, if w changes < tol stop the the algorithm 

In [18]:
brr_model.fit(Xreg, yreg)
brr_preds = brr_model.predict(Xreg_val)
r2_score(yreg_val, brr_preds)

0.4700037663317521